## Recap of Last Lecture:

- In the last class we began by introducing the field of **Computer Vision**, it's wide-ranging applications and reflecting on why it's hard!
- We then saw how adding CNN Blocks inplace of MLPs improved overall performance of our model on an image dataset. 
- After that, we delved into the intuition behind CNNs, it's various components and how each of them worked.

## Agenda for this Lecture:

- This lecture is divided into two parts: In Part 1,  we will **code a Convolution Block from scratch** and in Part 2, we will see ways to deal with the **Issue of Overfitting** in CNNs.
- Part 1 will be focussed on understanding the working of **Convolution** and **Pooling** functions by implementing it from scratch.
- In Part 2 we will broadly cover two classes of regularization techniques, 
    1. Modifications in **Model Pipeline** (architecture, loss functions etc.) 
    2. Modifications in **Data Pipeline** (data augmentations)

# Part 1. Convolution Block From Scratch

In this segment, we will extend our understanding about Convolution and MaxPooling by implementing them from scratch (using Numpy). Using some predefined filters, we will then apply convolution operation on a sample image.

### Setup (Import Statements)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import skimage
from skimage.io import imread, imshow
from skimage.transform import rescale, resize

### 1. Implementation of Conv Block (Convolution + Non-Linearity + Pooling) using Numpy

- Convolution:
    - In this function, we take in an image and a kernel (2D matrix), along with two optional parameters; padding and strides (both integers) and return the convolved image.
    - We iterate through every pixel in the image, and taking it as the central pixel, we perform element-wise multiplication of the surrounding pixels with the kernel pixels; finally replacing it with the net sum of those values.

- Non-Linearity:
    - We use ReLU as our non-linear function and apply it to every pixel in the image.
    - Notice that here we use a vectorized and more efficient implementation.
- Pooling:
    - We implement MaxPooling, in much the same way as Convolution, except that this time, we don't apply kernel over the image, rather a max function over a region of image surrounding the central pixel. That region is determined by the `size` parameter.

In [ ]:
# Convolution function
def convolve2D(image, kernel, padding, strides):
    # Gather Shapes of Kernel + Image + Padding
    (xImgShape, yImgShape) = image.shape[: 2]
    (xKernShape, yKernShape) = kernel.shape[: 2]

    # Shape of Output Convolution
    xOutput = int(((xImgShape - xKernShape + 2 * padding) / strides) + 1)
    yOutput = int(((yImgShape - yKernShape + 2 * padding) / strides) + 1)
    output = np.zeros((xOutput, yOutput))
  
    # Apply Equal Padding to All Sides
    if padding != 0:
        imagePadded = np.zeros((image.shape[0] + padding * 2, image.shape[1] + padding * 2))
        imagePadded[int(padding):int(-1 * padding), int(padding) : int(-1 * padding)] = image
        # print(imagePadded)
    else:
        imagePadded = image

    # Iterate through image
    for y in np.arange(0, image.shape[1], strides):
        # Exit Convolution
        if y > image.shape[1] - yKernShape:
            break
        # Only Convolve if y has gone down by the specified Strides
        if y % strides == 0:
            for x in range(0, image.shape[0], strides):
                # Go to next row once kernel is out of bounds
                if x > image.shape[0] - xKernShape:
                    break
                try:
                    # Only Convolve if x has moved by the specified Strides
                    if x % strides == 0:
                        output[x, y] = (kernel * imagePadded[x : x + xKernShape, y : y + yKernShape]).sum()
                except:
                    break

    return output

# Non-linearity (ReLU) function
def relu(feature_map):
    relu_out = np.where(feature_map > 0, feature_map, 0)
    return relu_out

# Pooling (MaxPool) function
def pooling(feature_map, size, stride):
    # Shape of pooling
    xOutput = int(((feature_map.shape[0] - size) / stride) + 1)
    yOutput = int(((feature_map.shape[1] - size) / stride) + 1)
    pool_output = np.zeros((xOutput, yOutput))

    # Iterate through image
    x2 = 0
    for y in np.arange(0,feature_map.shape[1],stride):
        # Exit Convolution
        if y > feature_map.shape[1] - size:
            break
        y2 = 0
        # Only Convolve if y has gone down by the specified Strides
        if y % stride == 0:
          for x in np.arange(0, feature_map.shape[0], stride):
              # Go to next row once kernel is out of bounds
              if x > feature_map.shape[0] - size:
                  break
              try:
                  # Only Convolve if x has moved by the specified Strides
                  if x % stride == 0:
                      pool_output[x2, y2] =  np.max([feature_map[y : y+size,  x : x+size]])
                      y2 += 1
              except:
                  break
          x2 += 1

    return pool_output

### 2. Common Filters
- In Lecture 1, we had already looked at some of the common filters, namely Horizontal and Vertical Filter.  
    - **Horizontal Filter** -
        \begin{bmatrix}
        1&   0&  -1\\ 
        1&   0&  -1\\ 
        1&   0&  -1
        \end{bmatrix}
    - **Vertical Filter** -
        \begin{bmatrix}
        1&  1&  1\\ 
        0&  0&  0\\ 
        -1&  -1&  -1
        \end{bmatrix}

- In this lecture, we will introduce two new filters and operate with it; namely,
    1. **Sharpen Filter** - As the name suggests, it is used to enhance the edges of objects and adjust the contrast and the shade characteristics in a image, with a matrix like the one shown below:
<br>
\begin{bmatrix}
0&  -1&  0\\ 
-1&  5& -1\\ 
0&  -1& 0
\end{bmatrix}
    2. **Blur Filter** - It has the effect of smoothening of image, by averaging all pixels surrounding a central pixel and replacing it with the averaged value. A 3x3 average filter is shown below:
    <br>
\begin{bmatrix}
\frac{1}{9}&  \frac{1}{9}&  \frac{1}{9}\\ 
\frac{1}{9}&  \frac{1}{9}&  \frac{1}{9}\\ 
\frac{1}{9}&  \frac{1}{9}&  \frac{1}{9}
\end{bmatrix}

In [ ]:
# construct average blurring kernels used to smooth an image
blur_filter = np.ones((3, 3), dtype='float') * (1.0 / (3 * 3))


# construct a sharpening filter
sharpen_filter = np.array((
     [0, -1,  0],
     [-1, 5, -1],
     [0, -1,  0]), dtype='int')

kernelBank = [("blur",blur_filter), ("sharpen",sharpen_filter)]

### 3. Apply Conv Block on a Sample Image using above Filters


In [ ]:
img = skimage.data.chelsea()
# Converting the image into gray.
gray = skimage.color.rgb2gray(img)
gray = resize(gray, (252, 252))

for (kernelName, K) in kernelBank:
    print("[INFO] applying {} kernel".format(kernelName))
    output = convolve2D(gray, K, 0, 1)
    act = relu(output)
    pool = pooling(act, size=2, stride=2)

    f, axarr = plt.subplots(1, 4, figsize=(12, 12))
    axarr[0].imshow(gray, cmap='gray')
    axarr[1].imshow(output, cmap='gray')
    axarr[2].imshow(act, cmap='gray')
    axarr[3].imshow(pool, cmap='gray')

    axarr = axarr.ravel()
    axarr[0].set_title("Original image")
    axarr[1].set_title(kernelName)
    axarr[2].set_title("relu")
    axarr[3].set_title("pooling")

- In the above example, we saw how different functions in Conv-Block operate on an Image and what visual changes they create.
- In other words, we showed how Forward Propagation actually works in a Conv-Block with the help of predefined filters.
- In the next lecture, we will implement both forward and backward propagation from scratch and train it on a custom dataset.  

But for now, let's move on to the next Part, which is about dealing with Overfitting in CNNs.


#Part 2. Tackling Overfitting

There are broadly two ways in which we can reduce overfitting -

- #### ***1. Modifications in Model Pipeline:***  Improving ***Network*** Configuration to solve over-parametrization
- #### ***2. Modifications in Data Pipeline:***  Augmenting ***Dataset*** to increase the number of samples

**But before even talking about these techniques, let's revisit the problem statement and the baseline model we developed in the last lecture** - 

### Setup 

#### Download Data

We are going to use the same collection of real-world clothing images dataset as in the first lecture, opensourced [here](https://github.com/alexeygrigorev/clothing-dataset-small).  

You can read more about the dataset in this [blog post](https://medium.com/data-science-insider/clothing-dataset-5b72cd7c3f1f).

In [ ]:
# !gdown https://drive.google.com/uc?id=1XdXz0TKo_KCDRHOMvzV-YtcTx7NPG-jC

In [ ]:
# !unzip /content/clothing-dataset-small.zip

#### Import Statements

In [ ]:
import os
import numpy as np
import tensorflow as tf
tf.keras.utils.set_random_seed(111)
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics
%matplotlib inline

## 0. Revisiting CNNs

Let's start from where we left off- 

[A typical CNN training pipeline]

- Problem Type: Image Classification
- Dataset: Clothing Dataset, 10 classes
- Model: 2-block (Conv+Maxpool) CNN model
    - Performance: ~50% accuracy on test dataset

These results are better than MLPs but far from perfect.   
In this module, we will investigate the reasons for poor performance and seek solution to improve the accuracy of our model.

Let's start with this as our **Baseline Model** and iteratively improve upon it to build a highly performant system. 

[Poll] Random Guess how much improvement we will be able to make (in terms of test accuracy) and in how many iterations (number of modifications)?

### Baseline Model

This is the same CNN model which we used in the previous module to train our clothing dataset.

#### Loading Data

In [ ]:
import tensorflow as tf
import os

In [ ]:
def load_data(base_dir = "/Users/debnsuma/Downloads/clothing-dataset-small"):

    assert os.path.exists(f"{base_dir}/train") and os.path.exists(f"{base_dir}/validation") and os.path.exists(f"{base_dir}/test")

    print("Loading Data...")
    train_data_path = os.path.join(base_dir, "train")
    train_data = tf.keras.utils.image_dataset_from_directory(train_data_path, shuffle=True, label_mode='categorical')

    val_data_path = os.path.join(base_dir, "validation")
    val_data = tf.keras.utils.image_dataset_from_directory(val_data_path, shuffle=False, label_mode='categorical')

    test_data_path = os.path.join(base_dir, "test")
    test_data = tf.keras.utils.image_dataset_from_directory(test_data_path, shuffle=False, label_mode='categorical')

    return train_data, val_data, test_data, train_data.class_names


In [ ]:
train_data, val_data, test_data, class_names = load_data()

In [ ]:
num_class = len(train_data.class_names)

#### Data Preprocessing: Resizing, Standardization

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def preprocess(train_data, val_data, test_data, target_height=128, target_width=128):

    # Data Processing Stage with resizing and rescaling operations
    data_preprocess = keras.Sequential(
        name="data_preprocess",
        layers=[
            layers.Resizing(target_height, target_width),
            layers.Rescaling(1.0/255),
        ]
    )

    # Perform Data Processing on the train, val, test dataset
    train_ds = train_data.map(lambda x, y: (data_preprocess(x), y), num_parallel_calls=tf.data.AUTOTUNE)
    val_ds = val_data.map(lambda x, y: (data_preprocess(x), y), num_parallel_calls=tf.data.AUTOTUNE)
    test_ds = test_data.map(lambda x, y: (data_preprocess(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = preprocess(train_data, val_data, test_data)

In [ ]:
type(train_ds)

#### Model Architecture

In [ ]:
def baseline(height=128, width=128):
    num_classes = 10
    hidden_size = 256

    model = keras.Sequential(
        name="model_cnn",
        layers=[
            layers.Conv2D(filters=16, kernel_size=3, padding="same", activation='relu', input_shape=(height, width, 3)),
            layers.MaxPooling2D(),
            layers.Flatten(),
            layers.Dense(units=hidden_size, activation='relu'),
            layers.Dense(units=num_classes, activation='softmax')
        ]
    )
    return model

In [ ]:
model = baseline()
model.summary()

#### Compile and Train

In [ ]:
import tensorflow as tf
from tensorflow import keras

def compile_train_v1(model, train_ds, val_ds, ckpt_path="checkpoint1.weights.h5"):
    epochs = 10
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model_fit = model.fit(train_ds, 
                          validation_data=val_ds, 
                          epochs=epochs, 
                          callbacks=[
                              keras.callbacks.ModelCheckpoint(ckpt_path, 
                                                              save_weights_only=True, 
                                                              monitor='val_accuracy', 
                                                              mode='max', 
                                                              save_best_only=True),
                          ])
    return model_fit

# Assuming model, train_ds, and val_ds are defined elsewhere in your code
model_fit = compile_train_v1(model, train_ds, val_ds)


#### Plot Train and Validation Accuracy

In [ ]:
# helper function to annotate maximum values in the plots
def annot_max(x,y, xytext=(0.94,0.96), ax=None, only_y=True):
    xmax = x[np.argmax(y)]
    ymax = max(y)
    if only_y:
        text = "{:.2f}%".format(ymax)
    else:
        text= "x={:.2f}, y={:.2f}%".format(xmax, ymax)
    if not ax:
        ax=plt.gca()
    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    arrowprops=dict(arrowstyle="->",connectionstyle="angle,angleA=0,angleB=60")
    kw = dict(xycoords='data',textcoords="axes fraction",
              arrowprops=arrowprops, bbox=bbox_props, ha="right", va="top")
    ax.annotate(text, xy=(xmax, ymax), xytext=xytext, **kw)

def plot_accuracy(model_fit):
    #accuracy graph
    x = range(0,len(model_fit.history['accuracy']))
    y_train = [acc * 100 for acc in model_fit.history['accuracy']]
    y_val = [acc * 100 for acc in model_fit.history['val_accuracy']]

    plt.plot(x, y_train, label='Train', color='b')
    annot_max(x, y_train, xytext=(0.7,0.9))
    plt.plot(x, y_val, label='Val', color='r')
    annot_max(x, y_val, xytext=(0.8,0.7))
    plt.ylabel('Accuracy', fontsize=15)
    plt.xlabel('epoch', fontsize=15)
    plt.legend()
    plt.show()

In [ ]:
plot_accuracy(model_fit)

#### Analyze Result on Test Dataset

In [ ]:
def print_accuracy_stats(model, ds, class_names):
    model.load_weights("checkpoint1.weights.h5")
    true_onehot = tf.concat([y for x, y in ds], axis=0)
    true_categories = tf.argmax(true_onehot, axis=1)
    y_pred = model.predict(ds)
    predicted_categories = tf.argmax(y_pred, axis=1)

    test_acc = metrics.accuracy_score(true_categories, predicted_categories) * 100
    print(f'\nTest Accuracy: {test_acc:.2f}%\n')

# Note: This doesn't work with shuffled datasets
def plot_confusion_matrix(model, ds, class_names):
    model.load_weights("checkpoint1.weights.h5")
    true_onehot = tf.concat([y for x, y in ds], axis=0)
    true_categories = tf.argmax(true_onehot, axis=1)
    y_pred = model.predict(ds)
    predicted_categories = tf.argmax(y_pred, axis=1)
    cm = metrics.confusion_matrix(true_categories,predicted_categories) # last batch 
    sns.heatmap(cm, annot=True, xticklabels=class_names, yticklabels=class_names, cmap="YlGnBu", fmt='g')
    plt.show()

In [ ]:
print_accuracy_stats(model, test_ds, class_names)
plot_confusion_matrix(model, test_ds, class_names)

#### What is the most apparent issue with the plots above?
**Validation Accuracy very low as compared to Training Accuracy**
(better known as Overfitting)

#### How do we solve the Overfitting in ML?

Overfitting = More Trainable Parameters, Less Training Samples
***Solutions -***
1. More Data, Data Augmentation
2. Reducing Complexity of Model Architecture (Smaller Model)
3. Adding Regularizer to Model
4. Adding Regularizer to Loss Function

## **1.** Modifications in Model Pipeline

The baseline network has ~17 million parameters and just over 3000 images for training. This results in overparamterization of the network making it prone to overfit. One way to tackle this would be to remove some layers from the network. But since it's already a small network, that would badly affect the performance on both training/validation set (You can try yourself to see that!). So what else can we do?

***HINT: Look at `model.summary()` carefully!***

We find that over ~99% of the parameters are associated with just one layer, and that's not even the Convolutional layer. Although we introduced CNN in the model architecture, the number of trainable parameters as a result of that is just ~5000 which is 1000 times less than that of the Dense Layers in the architecture.

The reason for it is that when we flatten the 3D matrix of shape 32x32x32 into 1D, the number of connections from that layer to dense layer explodes exponentially (no weight sharing in MLP).

In order to avoid this, we will try do two things:
1. Reduce the 3D volume before the flatten layer by using more Convolution blocks.
2. Replace the flatten layer by `GlobalAveragePooling2D`  

### **Modification #1**

In [ ]:
def arch_1(height=128, width=128):
    num_classes = 10
    hidden_size = 256

    model = keras.Sequential([
        layers.Conv2D(filters=16, kernel_size=3, padding="same", activation='relu', input_shape=(height, width, 3), name="conv2d_1"),
        layers.MaxPooling2D(name="maxpool2d_1"),
        layers.Conv2D(filters=32, kernel_size=3, padding="same", activation='relu', name="conv2d_2"),
        layers.MaxPooling2D(name="maxpool2d_2"),
        layers.Conv2D(filters=64, kernel_size=3, padding="same", activation='relu', name="conv2d_3"),
        layers.MaxPooling2D(name="maxpool2d_3"),
        layers.Conv2D(filters=128, kernel_size=3, padding="same", activation='relu', name="conv2d_4"),
        layers.MaxPooling2D(name="maxpool2d_4"),
        layers.Conv2D(filters=256, kernel_size=3, padding="same", activation='relu', name="conv2d_5"),
        layers.GlobalAveragePooling2D(name="global_avg_pool"),
        layers.Dense(units=hidden_size, activation='relu', name="dense_1"),
        layers.Dense(units=num_classes, activation='softmax', name="output")
    ], name="model_cnn")
    
    return model

In [ ]:
model = arch_1()
model.summary()

In [ ]:
def compile_train_v1(model, train_ds, val_ds, epochs=10, ckpt_path="checkpoint2.weights.h5"):
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    model_fit = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[
        keras.callbacks.ModelCheckpoint(ckpt_path, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True),
    ])
    return model_fit

In [ ]:
model_fit = compile_train_v1(model, train_ds, val_ds)

In [ ]:
plot_accuracy(model_fit)


**Unbelievable!**  
With just two modifications in the architecture, we can see the model performing much better and the problem of overfitting vanish.  

This is good news, but something seems not right about the above plots!   
#### **Can you guess what is wrong with the above plots?**  

It seems the model was still improving when we stopped training leading to ***underfitting***!  
So, let's continue training for more number of epochs. **But how many more epochs?**  

The answer is that *we do not know*. But perhaps we can guess, by looking at the training plots. **Can you?**   

- We can stop when the val loss/accuracy curve flattens out (or do not improve any further)!  
- This technique is better known as "early stopping" and `keras` provides a readymade function for that. 
- Effectively, it stops model training, once the condition set by `EarlyStopping` is met.
- Conditions could be based on either validation loss or accuracy numbers.
- The `patience` argument determines that after how many "epochs with no improvement", training should be stopped.

Let's plugin this into our training pipeline and see if it improves the result. [not implemented yet!]

In [ ]:
model_fit = compile_train_v1(model, train_ds, val_ds, epochs=30)

In [ ]:
plot_accuracy(model_fit)

In [ ]:
print_accuracy_stats(model, test_ds, class_names)
plot_confusion_matrix(model, test_ds, class_names)

We see a **MASSIVE IMPROVEMENT** in the performance on test dataset, but is this the best number we can achieve, using architecture improvements?
- Most Likely, the answer is No. Many improvements on the architecture side exists and we will look into some of them in subsequent lectures. (Also, heard of NAS?)
- But before moving on, let's try one more thing in our model. We already know from previous lectures that adding Dropout and BatchNorm helps in regularizing the model, thus improving performance on validation dataset. (One question on BN, Dropout; does the introduce new "training" parameters?) Let's see it in action next.

### **Modification #2** 
1. Add Batch Normalization after every `Conv2D` and `Dense` layers.
2. Add `Dropout` after `Dense` layers

#### **Q. What is Dropout and why is it required?**
- Drops out random neurons from the network during training
- Dropout reduces overfitting by :
    - Reducing over - reliability of the network on using certain features to identify in a image
    - Dropout thus encourages the network to utilize all the features to find pattern in a image
    - Thus increasing overall generalization of our model, thus giving good model performance


- Dropout is only used during training and not while evaluating our model as
    - We want to use the capability of every learned neurons and really don't like to skip some of them randomly.

<center><img src='https://drive.google.com/uc?id=1P0w9g-BuTHUCWQ6Xcz7A_lei_bQfdTX-' height=280></center>

#### **Q. What is Batch Normalization?**
- Simply put, it normalizes the inputs to each layer to a learnt representation likely close to μ = 0.0, σ = 1.0 wrt every minibatch.
- It's a four-step process as described below:

<center><img src='https://t3638486.p.clickup-attachments.com/t3638486/c04a328b-8726-4bce-9f8d-b121e0e157ba/image.png' height=200>
<img src='https://drive.google.com/uc?id=12DwZw4uWID62jT6PJHlmzhKeEe9plhxe' height=300></center>

#### **Q. Why Batch Normalization?**

- In simple terms, just like input normalization, if all the layer inputs are normalized, then the significant outliers are less likely to impact the training process in a negative way. 
- And if they do, their impact will be much lower than without using Batch Normalization.
- This inturn makes model less sensitive to hyperparameter tuning (for eg: larger LRs won't affect much the training process).

#### **Q. How Batch Normalization Behaves during Inference?**

- When inferring e.g. the class for a new sample, you wish to normalize it based on the entire training set, as it produces better estimates and is computationally feasible.

- Hence, during inference, the Batch Normalization step goes as follows:
<center><img src='https://t3638486.p.clickup-attachments.com/t3638486/e58670bb-997f-4acb-9909-534ec0536e1d/image.png' height=100></center>, where, 𝑥∈𝑋  and 𝑋 represents the full training data.

#### **Q. Now that we have understood Dropout and BN, where should we add it to our model?**

- Emperically it has been found out that dropout works best when it is placed in-between Dense layers of an architecture with a probability of 0.5.
- Sometimes adding dropout layers after MaxPooling Layer with lower probability (0.1 to 0.25) also enhances performance.
- These are not hard-set rules, rather standard practices which people tend to follow.

- As for BatchNorm, the original paper suggested to add BN layer just after Conv Layer and before non-linearity, but later-on it was experimentally found out that placing BN layers after non-linear layers worked better. 

- Other reasoning for placing BN after both Conv and non-linear layers like ReLU comes from Statistics. If we normalize the feature distribution right after Conv Layer and pass it to the ReLU block, it will clamp the negative values from BN output to 0, effectively changing the distribution itself and defeating the purpose of normalization.




In [ ]:
def arch_2(height=128, width=128):
    num_classes = 10
    hidden_size = 256

    model = keras.Sequential(
        name="model_cnn_2",
        layers=[
            layers.Conv2D(filters=16, kernel_size=3, padding="same", input_shape=(height, width, 3)),
            layers.Activation("relu"),
            layers.BatchNormalization(),
            layers.MaxPooling2D(),
            layers.Conv2D(filters=32, kernel_size=3, padding="same"),
            layers.Activation("relu"),
            layers.BatchNormalization(),
            layers.MaxPooling2D(),
            layers.Conv2D(filters=64, kernel_size=3, padding="same"),
            layers.Activation("relu"),
            layers.BatchNormalization(),
            layers.MaxPooling2D(),
            layers.Conv2D(filters=128, kernel_size=3, padding="same"),
            layers.Activation("relu"),
            layers.BatchNormalization(),
            layers.MaxPooling2D(),
            layers.Conv2D(filters=256, kernel_size=3, padding="same"),
            layers.Activation("relu"),
            layers.BatchNormalization(),
            # layers.MaxPooling2D(),
            # layers.Flatten(),
            layers.GlobalAveragePooling2D(),
            layers.Dense(units=hidden_size),
            layers.Activation("relu"),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            layers.Dense(units=num_classes, activation='softmax')
        ]
    )
    return model

In [ ]:
model = arch_2()
model.summary()

In [ ]:
model_fit = compile_train_v1(model, train_ds, val_ds, epochs=30)

In [ ]:
plot_accuracy(model_fit)

**WAIT, WAIT..SOMETHING AIN'T RIGHT HERE...**   

We see the training is not very stable here (too many fluctuations)! This was "kinda" expected since we added more chaos to the training process, for example "dropout" switches off 20% of connections randomly at every step. 


*Note: Every regularization technique induces some form of chaos to the training process.*  

#### Q.**How to solve this issue?**

**Rule of Thumb:** Fluctuating (heartbeat) plots imply that the learning rate is too high.  

So, we can try tuning it. Or a better alternative is to use a Learning Rate Scheduler and let the model train until the loss/accuracy curve flattens out (also called Early Stopping). 

In Keras, we can simply add both LR Scheduler and Early Stopping as a callback to `model.fit`.

<br><br>

# **Quiz 1:** Batch Normalization is helpful in overfitting because :
	
a) It normalizes all inputs before sending it to the next layer<br>
b) It returns back the normalized mean and standard deviation of weights<br>
c) It is a very efficient backpropagation technique<br>
d) None of the above<br>

Ans : a)

Batch normalization is a process to make neural networks faster and more stable through adding extra layers in a deep neural network. The new layer performs the normalizing operation on the input of a layer coming from a previous layer.
It works as a regularization technique by adding some noise to the data and thus also prevents overfitting

<br><br>

### **Modification #3** 

#### 1. Use `ReduceLROnPlateau` as LR Scheduler and `EarlyStopping` to stop training once the loss curve plateaus.


In [ ]:
def compile_train_v2(model, train_ds, val_ds, epochs=10, ckpt_path="/tmp/checkpoint"):
    callbacks = [
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.3, patience=5, min_lr=0.00001
        ),
        keras.callbacks.ModelCheckpoint(ckpt_path, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True),
        keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10, min_delta=0.001, mode='min'
        )
    ]
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    model_fit = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)
    return model_fit

In [ ]:
model = arch_2()
model_fit = compile_train_v2(model, train_ds, val_ds, epochs=100)

In [ ]:
plot_accuracy(model_fit)

In [ ]:
print_accuracy_stats(model, test_ds, class_names)
plot_confusion_matrix(model, test_ds, class_names)

**Nice Improvements!**

Before moving on, let's also study the effect of adding L1/L2 Regularizer to loss functions.   
- L2 regularization reduces the impact of insignificant features, thereby, giving a generalizable model. L1 regularization removes some of the features from the model to reduce overfitting.
- In `Keras`, we add regularizers in the model definition itself.

#### Q. **What is L1 Regularization ?** 
L1 regularization, also known as L1 norm, combats overfitting by shrinking the parameters towards 0, hence making some features obsolete

- L1 Regularization can be interpreted as a form of feature selection
- Because when we assign a feature weight = 0, we remove the significance of the feature
-  If the input features of our model have weights closer to 0, our L1 norm would be sparse. 
- A selection of the input features would have weights equal to zero, and the rest would be non-zero. 

To understand L1 Regularization better, lets take an example : <br>
Supoose we want to estimate car prices using machine learning, using the features :
- Brand
- Engine power
- Fuel capacity
- Year of manufacturing
- Cabin volume

When predicting the price of a car, different features will have difference infleunce on the price. For example, it is likely that Fuel capacity of car has a lower influence on the price of a car than Engine power.


So our L1 Regularization technique would assign Fuel capacity feature with a zero weight, because it does not have significant effect on the price. We can expect Engine power and other infuential features on the car price to be assigned non-zero weights.

Mathematically L1 Regularization can be represented as :

<img src='https://drive.google.com/uc?id=10kKlUkyr849jz9bzz_5Wcb55DvK1mvKc' height=200>

Essentially, when we use L1 regularization, we are penalizing the absolute value of the weights. 

L1 Regularizaiton would produce a model that is highly generalized, and only uses a subset of input features, thus reducing the complexity of the model, thus reducing overfitting. 

#### Q. **What is L2 Regularization ?** 

- L2 Regularizaiton or L2 norm, counters overfitting by forcing weights to be small, but not making them exactly 0.

- So if we are predicting car prices again, this means that less significant features for predicting the car prices would still have some influence over the final prediction, but it would only be a small influence. 

The regularization term that we add to the loss function when performing L2 regularization is the sum of squares of all of the feature weights:

<img src='https://drive.google.com/uc?id=1Be9BvuJDV6gzO8PJtwZjMz3blG6pLczd' height=70>


So, L2 regularization returns a non-sparse solution since the weights will be non-zero (although some may be close to 0).


### **Modification #4** 
#### 1. Add L2 regularization to `Conv2D` and `Dense` layers




In [ ]:
def arch_3(height=128, width=128):
    num_classes = 10
    hidden_size = 256

    model = keras.Sequential(
        name="model_cnn_3",
        layers=[
            layers.Conv2D(filters=16, kernel_size=3, padding="same", input_shape=(height, width, 3),
                            kernel_regularizer=regularizers.l2(1e-3)),
            layers.Activation("relu"),
            layers.BatchNormalization(),
            layers.MaxPooling2D(),
            layers.Conv2D(filters=32, kernel_size=3, padding="same",
                            kernel_regularizer=regularizers.l2(1e-3)),
            layers.Activation("relu"),
            layers.BatchNormalization(),
            layers.MaxPooling2D(),
            layers.Conv2D(filters=64, kernel_size=3, padding="same",
                            kernel_regularizer=regularizers.l2(1e-3)),
            layers.Activation("relu"),
            layers.BatchNormalization(),
            layers.MaxPooling2D(),
            layers.Conv2D(filters=128, kernel_size=3, padding="same",
                            kernel_regularizer=regularizers.l2(1e-3)),
            layers.Activation("relu"),
            layers.BatchNormalization(),
            layers.MaxPooling2D(),
            layers.Conv2D(filters=256, kernel_size=3, padding="same",
                            kernel_regularizer=regularizers.l2(1e-3)),
            layers.Activation("relu"),
            layers.BatchNormalization(),
            # layers.MaxPooling2D(),
            # layers.Flatten(),
            layers.GlobalAveragePooling2D(),
            layers.Dense(units=hidden_size, kernel_regularizer=regularizers.l2(1e-3)),
            layers.Activation("relu"),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            layers.Dense(units=num_classes, activation='softmax')
        ]
    )
    return model

In [ ]:
model = arch_3()
model.summary()

In [ ]:
model_fit = compile_train_v2(model, train_ds, val_ds, epochs=100)

In [ ]:
plot_accuracy(model_fit)

In [ ]:
print_accuracy_stats(model, test_ds, class_names)
plot_confusion_matrix(model, test_ds, class_names)

**Well, we can see that after adding regularization, the test accruacy improved by quite a bit!**

# **Quiz 2 :** Which of these is true ?

L1 and L2 regularization are techniques to reduce the model complexity and prevent overfitting.
L2 regression shrinks the coefficients
L1 regression not only helps in reducing overfitting but it can help us in feature selection
All of the above

Ans : d)

By adding regularization terms, the value of weights reduces by assuming that a neural network having less weights makes simpler models. And hence, it reduces the overfitting to a certain level.<br>
L1 regularization, also known as L1 norm or Lasso (in regression problems), combats overfitting by shrinking the parameters towards 0. This makes some features obsolete. It’s a form of feature selection, because when we assign a feature with a 0 weight, we’re multiplying the feature values by 0 which returns 0, eradicating the significance of that feature.


## **2.** Modifications in Data Pipeline (Data Augmentation)

#### What is Augmentation?

- Augmentation is a technique to `artificially increase` the amount of data by generating new data points from `existing data`. 

**How can we increase the data from existing data?**  

- By making Minor changes such as `flips, translations,brigntness,coloring, rotations etc`. 

<img src='https://drive.google.com/uc?id=15SJ7_ImxN3Ouzdx6T921JtFg0mS0P_Z4' height=200>



**How does it help with training?**  

- This leads to greater diversity of data samples being seen by the network hence decreasing the likelihood of overfitting the model on the training dataset. 

- Also it helps in reducing some of the spurious characteristics of the dataset.

#### What different sort of Augmentation strategies we can apply?

Some most common data augmentations are:
* padding
* random rotating
* re-scaling,
* vertical and horizontal flipping
* translation ( image is moved along X, Y direction)
* cropping
* zooming
* darkening & brightening/color modification
* grayscaling
* changing contrast
* adding noise
* random erasing

We will discuss some of them in detail here, along with their code in `Keras`.

#### How do we apply augmentation in `tensorflow` / `keras`?

There are many ways to apply augmentation in Tensorflow/Keras, few of them are discussed here:
1. using the Keras Preprocessing Layers, just like preprocessing functions like `resizing` and `rescaling`, keras also provides data augmentation layers like `tf.keras.layers.RandomFlip`,  `tf.keras.layers.RandomRotation`, etc. These can be used in a similar way as we used the preprocessing functions.
2. using tf.image methods like `tf.image.stateless_random_flip_up_down`, `tf.image.stateless_random_brightness`
3. using Keras ImageDataGenerator API - It provides quick, out-of-the-box suite of augmentation techniques like standardization, rotation, shifts, flips, brightness change, and many more. 

In this module, we will be using mostly option 1. Keras Preprocessing Layers for data augmentation.

Load Samples

In [ ]:
# Functions to Load and Visualize Samples
# Load Sample
def load_sample(output_size=(300, 300)):
    sample = keras.utils.get_file(
        "sample.jpg", 
        "https://t3638486.p.clickup-attachments.com/t3638486/a0a4f633-2963-4e06-acae-2ad1d12eba3e/sample.jpg",
    )
    sample = keras.utils.load_img(sample, target_size=output_size)
    sample = keras.utils.img_to_array(sample)
    return sample

# Visualize Sample
def show_images(imgs, num_rows, num_cols, scale=2):
    # show augmented images in a grid layout 
    aspect_ratio = imgs[0].shape[0]/imgs[0].shape[1]
    figsize = (num_cols * scale, num_rows * scale * aspect_ratio)
    _, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
    for i in range(num_rows):
        for j in range(num_cols):
            if num_rows != 1:
                axes[i][j].imshow(imgs[i * num_cols + j].numpy().astype("uint8"))
                axes[i][j].axes.get_xaxis().set_visible(False)
                axes[i][j].axes.get_yaxis().set_visible(False)
            else:
                plt.imsave('test.png', imgs[i * num_cols + j].numpy()[0].astype("uint8"))
                # print(imgs[i * num_cols + j].numpy()[0].astype("uint8"))
                axes[j].imshow(imgs[i * num_cols + j].numpy().astype("uint8"))
                axes[j].axes.get_xaxis().set_visible(False)
                axes[j].axes.get_yaxis().set_visible(False)
    plt.subplots_adjust(hspace=0.1, wspace=0)
    return axes

def apply(img, aug, num_rows=1, num_cols=4, scale=3):
    # apply augmentation multiple times to obtain different samples
    Y = [aug(img) for _ in range(num_rows * num_cols)]
    show_images(Y, num_rows, num_cols, scale)

#### Basic Augmentation Techniques using `tf.keras.layers.*` and `tf.image`

All these augmentation techniques will only apply if the mode is set to `training`!

In [ ]:
# Required to reset the mode to training! 
tf.keras.backend.clear_session()

a. `RandomCrop`

During training, this layer will randomly choose a location to crop images down to a target size. The layer will crop all the images in the same batch to the same cropping location.

In [ ]:
from keras.layers import RandomCrop

sample = load_sample(output_size=(300, 300))
aug_layer = RandomCrop(
    height = 224,
    width = 224,
)

apply(sample, aug_layer)

In [ ]:
# Alternative Implementation using tf.image
random_crop = tf.image.random_crop(value=sample, size=(224, 224, 3))

b. `RandomTranslation` 

This layer will apply random translations to each image during training, filling empty space according to `fill_mode`.

In [ ]:
from keras.layers import RandomTranslation

sample = load_sample(output_size=(300, 300))
aug_layer = RandomTranslation(
    height_factor = (-0.2, 0.3),
    width_factor = (-0.2, 0.3),
)

apply(sample, aug_layer)

In [ ]:
# Alternative Implementation using tf.image
random_shift = tf.keras.preprocessing.image.random_shift(sample, hrg = 0.2, wrg = 0.5)

c. `RandomRotation`

This layer will apply random rotations to each image, filling empty space according to fill_mode.

In [ ]:
from keras.layers import RandomRotation

sample = load_sample(output_size=(300, 300))
aug_layer = RandomRotation(
    factor = (-0.2, 0.3),
)

apply(sample, aug_layer)

In [ ]:
# Alternative Implementation using tf.image
random_rotation = tf.keras.preprocessing.image.random_rotation(sample, rg=500, channel_axis = 2, row_axis=0, col_axis=1)

d. `RandomFlip`  

This layer will flip the images horizontally and or vertically based on the mode attribute.

In [ ]:
from keras.layers import RandomFlip

sample = load_sample(output_size=(300, 300))
aug_layer = RandomFlip()

apply(sample, aug_layer)

In [ ]:
# Alternative Implementation using tf.image
randomflip = tf.image.random_flip_left_right(sample, 10)

e. `RandomBrightness`

This layer will randomly increase/reduce the brightness for the input RGB images. 

In [ ]:
sample = load_sample(output_size=(300, 300))

try:
    from tensorflow.keras.layers import RandomBrightness
    aug_layer = RandomBrightness(factor=(-0.2, 0.5))
except ImportError:
    print("This Version of tensorflow doesn't have tensorflow.keras.layers.RandomBrightness class, using tf.image.random_brightness instead")
    aug_layer = lambda x: tf.image.random_brightness(x, 100)
apply(sample, aug_layer)

In [ ]:
# Alternative Implementation using tf.image
random_bright = tf.image.random_brightness(sample, 100)

f. `RandomContrast`

This layer will randomly adjust the contrast of an image or images by a random factor. Contrast is adjusted independently for each channel of each image during training.

In [ ]:
from keras.layers import RandomContrast

sample = load_sample(output_size=(300, 300))
aug_layer = RandomContrast(factor=(0.5, 0.9))

apply(sample, aug_layer)

In [ ]:
# Alternative Implementation using tf.image
random_contrast = tf.image.random_contrast(sample, 0.5, 10)


# **Quiz 3:** What are the advantages of data augmentation ?

a) It prevents underfitting.<br>
b) It is used to expand the dataset by variations and helps in developing a more generalized model<br>
c) Both a) and b)<br>
d) None of the above<br>

Ans : b)

Solution :

Data augmentation is useful to improve performance and outcomes of machine learning models by forming new and different examples to train datasets. If the dataset in a machine learning model is rich and sufficient, the model performs better and more accurately.


Now let's apply some of the augmentation techniques on the clothing-dataset!

### Modification #5: Apply Data Augmentation

In [ ]:
def preprocess_v2(train_data, val_data, test_data, target_height=128, target_width=128):

    # Data Processing Stage with resizing and rescaling operations #same as before for test,val
    data_preprocess = keras.Sequential(
        name="data_preprocess",
        layers=[
            layers.Resizing(target_height, target_width),
            layers.Rescaling(1.0/255),
        ]
    )

    # Data Processing Stage with resizing and rescaling operations
    data_augmentation = keras.Sequential(
        name="data_augmentation",
        layers=[
            layers.Resizing(156, 156), # First resize to 156,156
            layers.RandomCrop(target_height, target_width), # Then randomly crop 128,128 region
            # layers.RandomBrightness(0.2), # Modify brightness by 0.2 factor
            layers.Rescaling(1.0/255), # Finally rescale
        ]
    )

    # Perform Data Processing on the train, val, test dataset
    train_ds = train_data.map(
        lambda x, y: (data_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE
    ).prefetch(tf.data.AUTOTUNE)
    val_ds = val_data.map(
        lambda x, y: (data_preprocess(x), y), num_parallel_calls=tf.data.AUTOTUNE
    ).prefetch(tf.data.AUTOTUNE)
    test_ds = test_data.map(
        lambda x, y: (data_preprocess(x), y), num_parallel_calls=tf.data.AUTOTUNE
    ).prefetch(tf.data.AUTOTUNE)

    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = preprocess_v2(train_data, val_data, test_data)

In [ ]:
model = arch_3()
model.summary()

In [ ]:
model_fit = compile_train_v2(model, train_ds, val_ds, epochs=100)

In [ ]:
plot_accuracy(model_fit)

In [ ]:
print_accuracy_stats(model, test_ds, class_names)
plot_confusion_matrix(model, test_ds, class_names)

**>80% Val Accuracy**  

For the first time ever, we were able to breach 80% val accuracy barrier and also obtained highest ever test accuracy, which shows that Data Augmentation helps.

#### But does Data Augmentation always improve the performance of the model?
* The main idea **how Augmentation works is** `it improve model Performance when we have less data.`
* By increasing the amount of Data.
* So, that our **model can generalize better**.
* **But Data Augmentation improves result till a particular point only.**


# **Quiz 4:** Which of the following statements is true ?
	
a) Shallow neural networks have only one hidden layer<br>
b) Ridge regression is an example of L1 regularization<br>
c) Max Pooling layer doesn’t have any trainable parameters<br>
d) All of the above<br>

Ans : a), c)

Shallow neural networks have only one hidden layer, in contrast to deep neural networks which have multiple hidden layers.
Ridge regression is an example of L2 regularization. 

Lasso regression is an example of L1 regularization
True, Max Pooling layers don't have any trainable parameters. 

In the forward pass, it passes the maximum value within each rectangle to the next layer. 
In the backward pass, it propagates error in the next layer to the place where the max value is taken.


# Summary : 

In this lecture, we covered:

- How to Code Convolution and Max Pooling layers from scratch!
- Batch Normalization
- Dropout
- L1 and L2 Regularization
- Data Augmentation Techniques

In the next class, we will discuss about forward and backward propagation in CNN components.